# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
file_path = "../WeatherPy/output_data/cities.csv"

cities_df = pd.read_csv(file_path)
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,chuy,-33.6971,-53.4616,68,77,0,9.80,UY,1.611886e+09
1,monywa,22.1167,95.1333,69,57,0,5.82,MM,1.611886e+09
2,rikitea,-23.1203,-134.9692,79,67,23,5.37,PF,1.611886e+09
3,tiksi,71.6872,128.8694,-26,86,99,8.01,RU,1.611886e+09
4,airai,-8.9266,125.4092,64,100,100,7.61,TL,1.611886e+09
...,...,...,...,...,...,...,...,...,...
584,okhansk,57.7186,55.3877,15,86,0,8.95,RU,1.611886e+09
585,nizwa,22.9333,57.5333,51,64,0,1.28,OM,1.611886e+09
586,port dickson,2.5228,101.7959,77,79,11,5.30,MY,1.611886e+09
587,lingao,19.9103,109.6859,66,64,7,15.32,CN,1.611886e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
gmaps.configure(api_key=g_key)

In [12]:
locations = cities_df[['Lat','Lng']]
locations

,Lat,Lng
0,-33.6971,-53.4616
1,22.1167,95.1333
2,-23.1203,-134.9692
3,71.6872,128.8694
4,-8.9266,125.4092
...,...,...
584,57.7186,55.3877
585,22.9333,57.5333
586,2.5228,101.7959
587,19.9103,109.6859


In [13]:
humidity_cities = cities_df['Humidity']
humidity_cities
# cities_df['Humidity'].max()

0       77
1       57
2       67
3       86
4      100
      ... 
584     86
585     64
586     79
587     64
588     84
Name: Humidity, Length: 589, dtype: int64

In [14]:
# Plot Heatmap
fig = gmaps.figure(center=(0,0),zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_cities, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=5, opacity=0.6)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
cities_df.dtypes

City           object
Lat           float64
Lng           float64
Max Temp        int64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date          float64
dtype: object

In [17]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don’t contain all three conditions. You want to be sure the weather is ideal.

weather_fit_df = cities_df.loc[(cities_df['Max Temp'] < 80) & (cities_df['Max Temp'] > 70) & (cities_df['Wind Speed'] < 10) & (cities_df['Cloudiness'] == 0)]

weather_fit_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
51,saint-philippe,-21.3585,55.7679,77,65,0,8.05,RE,1.611886e+09
87,busselton,-33.6500,115.3333,73,63,0,1.99,AU,1.611886e+09
94,wamba,8.9333,8.6000,72,19,0,3.83,NG,1.611886e+09
159,sao filipe,14.8961,-24.4956,73,67,0,5.48,CV,1.611886e+09
175,vila velha,-20.3297,-40.2925,77,83,0,9.22,BR,1.611886e+09
328,esmeraldas,-19.7625,-44.3139,75,57,0,6.91,BR,1.611886e+09
372,saint-joseph,-21.3667,55.6167,77,65,0,8.05,RE,1.611886e+09
408,maua,-23.6678,-46.4614,78,57,0,2.30,BR,1.611886e+09
424,lamu,-2.2717,40.9020,76,87,0,9.95,KE,1.611886e+09
425,presidencia roque saenz pena,-26.7852,-60.4388,77,53,0,5.95,AR,1.611886e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
